## Leaf Spot - Lesion Counts and Area from Tissue Scans

In [ ]:
%matplotlib inline
import os
import sys
# sys.path.append('/home/user/plantcv')
import numpy as np
import pandas as pd
import cv2
import imutils
from matplotlib import pyplot as plt
from plantcv import plantcv as pcv
from tkinter import filedialog
import tkinter as Tk
from tkinter import *
from datetime import datetime
from tkinter.filedialog import asksaveasfilename
from pandas import ExcelWriter
from scipy.spatial import distance as dist

def show_pic(img, cmap="gray"):
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(111)
    ax.imshow(img)

def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

In [ ]:
root = Tk()
root.withdraw()
path = filedialog.askdirectory()

images = []
infected_area = []
lesion_count = []

for file in os.listdir(path):
    if not file.startswith('.') and '.jpg' in file:
        images.append(file)
        filename = path + "/" + file
        image = cv2.imread("%s"%(filename))

        num = 1
        pixel = 0
        alpha_level = list(round(i,1) for i in np.linspace(1,3,30))
        pixel_level = []

        while pixel < 150:
            try:
                alpha = alpha_level[num]
                adjusted_leaf = cv2.convertScaleAbs(image, alpha=alpha)
                pixel = adjusted_leaf[adjusted_leaf != 255].mean()
                pixel_level.append(pixel)
                num += 1
            except IndexError:
                pixel_value = 150
                num = pixel_level.index(min(pixel_level, key=lambda x:abs(x-pixel_value)))
                break

        adjusted_leaf_img = cv2.convertScaleAbs(image, alpha=alpha_level[num-1])
        leaf_image_blur = cv2.medianBlur(adjusted_leaf_img, 25)
        leaf_image_blur_gray = cv2.cvtColor(leaf_image_blur, cv2.COLOR_BGR2GRAY)
        leaf_image_res ,leaf_image_thresh = cv2.threshold(leaf_image_blur_gray,240,255,cv2.THRESH_BINARY_INV)
        leaf_cnts = cv2.findContours(leaf_image_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        leaf_cnts = imutils.grab_contours(leaf_cnts)

        leaf_area = []
        for contour in range(len(leaf_cnts)):
            leaf_area.append(cv2.contourArea(leaf_cnts[contour]))

        adjusted_ls_img = cv2.convertScaleAbs(image, alpha=alpha_level[num-1])
        ls_image_blur_gray = cv2.cvtColor(adjusted_ls_img, cv2.COLOR_BGR2GRAY)
        ls_image_res ,ls_image_thresh = cv2.threshold(ls_image_blur_gray,135,200,cv2.THRESH_BINARY_INV)
        ls_kernel = np.ones((2,2),np.uint8)
        ls_opening = cv2.morphologyEx(ls_image_thresh, cv2.MORPH_OPEN, ls_kernel)
        ls_dist_transform = cv2.distanceTransform(ls_opening, cv2.DIST_L2,3)
        ls_ret, ls_last_image =  cv2.threshold(ls_dist_transform, 0.10*ls_dist_transform.max(),255,0)
        ls_last_image = np.uint8(ls_last_image)
        ls_cnts = cv2.findContours(ls_last_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        ls_cnts = imutils.grab_contours(ls_cnts)

        lesion_area = []
        for contour in range(len(ls_cnts)):
            lesion_area.append(cv2.contourArea(ls_cnts[contour]))

        try:
#         print("Estimated Lesions per Leaf = ", len(ls_cnts)/len(leaf_cnts))
            lesion_count.append(len(ls_cnts)/len(leaf_cnts))
        except ZeroDivisionError:
            print("Blank Image")
            lesion_count.append(0)

        try:
#         print("Estimated Lesion Area Infected = ", (sum(lesion_area)/sum(leaf_area))*100)
            infected_area.append((sum(lesion_area)/sum(leaf_area))*100)
        except ZeroDivisionError:
            infected_area.append(0)

data = pd.DataFrame({'Images':images,'Leaf Spot Percent':infected_area,'Lesion Counts (Per Leaf)':lesion_count})
data['Plot'] = data['Images'].map(lambda x: x.split('_')[1].split('.')[0])

root = Tk()
root.withdraw()
writer = ExcelWriter(asksaveasfilename(defaultextension='.xlsx'))
root.update()
data[['Images','Plot','Leaf Spot Percent','Lesion Counts (Per Leaf)']].to_excel(writer, index=False)
writer.save()
root.destroy()